In [3]:
import pymongo
import numpy as np
import pandas as pd

In [4]:
# Connect with mongodb

try:
    with open("credentials.txt", 'r', encoding='utf-8') as f:
        [name,password,url,dbname]=f.read().splitlines()
        conn=pymongo.MongoClient("mongodb://{}:{}@{}/{}".format(name,password,url,dbname))
    print ("Connected successfully!!!")
except pymongo.errors.ConnectionFailure as e:
    print ("Could not connect to MongoDB: %s" % e) 
    

    
db = conn['tweempact']
collection = db['tweets']

Connected successfully!!!


In [11]:
db.collection_names()

C:\Users\Albert Prat\Anaconda3\lib\site-packages\ipykernel_launcher.py:1: DeprecationWarning: collection_names is deprecated. Use list_collection_names instead.
  """Entry point for launching an IPython kernel.


['system.indexes', 'some-tweets', 'tweets', 'tweet_users']

In [13]:
def read_mongo(db, collection, query={}, host='localhost', port=27017, username=None, password=None, no_id=True):
    """ Read from Mongo and Store into DataFrame """
    
    # Make a query to the specific DB and Collection
    cursor = db[collection].find(query)

    # Expand the cursor and construct the DataFrame
    df =  pd.DataFrame(list(cursor))

    # Delete the _id
    if no_id:
        del df['_id']

    return df

In [18]:
df = read_mongo(db, "tweets")

In [41]:
df.head(2)

,contributors,coordinates,created_at,entities,extended_entities,favorite_count,favorited,geo,id,id_str,...,quoted_status_id,quoted_status_id_str,retweet_count,retweeted,retweeted_status,source,text,truncated,user,withheld_in_countries
0,None,None,Sun Sep 30 15:58:44 +0000 2018,"{'hashtags': [{'text': 'FCJumilla', 'indices':...",NaN,18,False,None,1046429145914384386,1046429145914384386,...,NaN,NaN,4,False,NaN,"<a href=""http://twitter.com/download/android"" ...",Desde el #FCJumilla damos nuestra más sincera ...,True,"{'id': 2664945430, 'id_str': '2664945430', 'na...",NaN
1,None,None,Mon Sep 24 03:05:47 +0000 2018,"{'hashtags': [], 'symbols': [], 'user_mentions...",NaN,0,False,None,1044060298443919367,1044060298443919367,...,NaN,NaN,0,False,NaN,"<a href=""https://mobile.twitter.com"" rel=""nofo...",Me estoy volviendo loca hasta por cada una de ...,False,"{'id': 713359736, 'id_str': '713359736', 'name...",NaN


In [62]:
df["quoted_status_id"].count()

346

In [59]:
(df["retweeted"]==True).sum()

0

In [63]:
cleaned = df[["id", "created_at","entities", "extended_entities","favorite_count","retweet_count","source","text", "truncated","user","quoted_status_id"]]

In [92]:
ids = [user["id"] for user in cleaned["user"]]
cleaned["id_num"] = ids

C:\Users\Albert Prat\Anaconda3\lib\site-packages\ipykernel_launcher.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  


In [96]:
subset = cleaned[cleaned["id_num"]==ids[0]]

In [107]:
import datetime

In [110]:
datetime.datetime.strptime(subset["created_at"][0], "%Y")

ValueError: time data 'Sun Sep 30 15:58:44 +0000 2018' does not match format '%Y'